In [5]:
import pandas as pd

# original file headers
headers_orig = ['index', 'contributors', 'coordinates', 'created_at', 'display_text_range', 'entities', \
 'extended_entities', 'favorite_count', 'favorited', 'full_text', 'geo', 'id', 'id_str', \
 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', \
 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place', 'possibly_sensitive', 'quoted_status', \
 'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink', 'retweet_count', 'retweeted', \
 'retweeted_status', 'scopes', 'source', 'truncated', 'user', 'withheld_in_countries']

# modified headers to match the troll data
headers = ['index', 'contributors', 'coordinates', 'publish_date', 'display_text_range', 'entities', \
 'extended_entities', 'favorite_count', 'favorited', 'content', 'geo', 'id', 'id_str', \
 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', \
 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place', 'possibly_sensitive', 'quoted_status', \
 'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink', 'retweet_count', 'retweeted', \
 'retweeted_status', 'scopes', 'source', 'truncated', 'user', 'withheld_in_countries']

dtypes_full = {'index':'int32', 'contributors':'str', 'coordinates':'str', 'created_at':'str', 'display_text_range':'str', 'entities':'str', \
 'extended_entities':'str', 'favorite_count':'int32', 'favorited':'int32', 'full_text':'str', 'geo':'str', 'id':'int32', 'id_str':'str', \
 'in_reply_to_screen_name':'str', 'in_reply_to_status_id':'int32', 'in_reply_to_status_id_str':'str', 'in_reply_to_user_id':'int32', \
 'in_reply_to_user_id_str':'str', 'is_quote_status':'int32', 'lang':'str', 'place':'str', 'possibly_sensitive':'int32', 'quoted_status':'int32', \
 'quoted_status_id':'int32', 'quoted_status_id_str':'str', 'quoted_status_permalink':'str', 'retweet_count':'int32', 'retweeted':'int32', \
 'retweeted_status':'str', 'scopes':'int32', 'source':'str', 'truncated':'int32', 'user':'str', 'withheld_in_countries':'str'}

dtypes = {'quoted_status_id_str':'str'}

russian_data = pd.read_csv("../data/russian_tweets/IRAhandle_tweets_1.csv", index_col=None) 
election_data = pd.read_csv("../data/election_tweets/election-day-5.5k-tweets.csv", header=None, names=headers, dtype=dtypes)#, header=None, names=headers, dtype=dtypes)

c:\python37\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (1,7,11,12,14,15,17,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# drop unnecessary troll columns
russian_data = russian_data.drop(columns=['region', 'harvested_date', 'new_june_2018'])
russian_data = russian_data.fillna(0)
russian_data.head()

,external_author_id,author,content,language,publish_date,following,followers,updates,post_type,account_type,retweet,account_category
0,9.060000e+17,10_GOP,"""We have a sitting Democrat US Senator on tria...",English,10/1/2017 19:58,1052,9636,253,0,Right,0,RightTroll
1,9.060000e+17,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,English,10/1/2017 22:43,1054,9637,254,0,Right,0,RightTroll
2,9.060000e+17,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,English,10/1/2017 22:50,1054,9637,255,RETWEET,Right,1,RightTroll
3,9.060000e+17,10_GOP,JUST IN: President Trump dedicates Presidents ...,English,10/1/2017 23:52,1062,9642,256,0,Right,0,RightTroll
4,9.060000e+17,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",English,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll


In [ ]:
# Separate and standardize the features
from sklearn.preprocessing import StandardScaler

features = russian_data.columns.tolist()

x = russian_data.loc[:, features].values
x = StandardScaler().fit_transform(x)

In [ ]:
# perform PCA
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.model_selection import train_test_split

pca = PCA(n_components=2)

principalComponents = pca.fit_transform(x)

principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
finalDf = pd.concat([principalDf, df[['account_category']]], axis = 1)

print(pca.explained_variance_ratio_)

In [ ]:
# draw graph
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = ['RightTroll', 'LeftTroll']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['account_category'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()